<a href="https://colab.research.google.com/github/MParsaMo/CMAPSS_1DConv_LSTM/blob/main/CMAPSS_1DConv_LSTM_unified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing train1 files :
from google.colab import files
train1 = files.upload()

Saving RUL_FD001.txt to RUL_FD001.txt
Saving RUL_FD002.txt to RUL_FD002.txt
Saving RUL_FD003.txt to RUL_FD003.txt
Saving RUL_FD004.txt to RUL_FD004.txt
Saving test_FD001.txt to test_FD001.txt
Saving test_FD002.txt to test_FD002.txt
Saving test_FD003.txt to test_FD003.txt
Saving test_FD004.txt to test_FD004.txt
Saving train_FD001.txt to train_FD001.txt
Saving train_FD002.txt to train_FD002.txt
Saving train_FD003.txt to train_FD003.txt
Saving train_FD004.txt to train_FD004.txt


In [ ]:
import os

# List all files in the current directory
print(os.listdir())


['.config', 'train_FD004.txt', 'train_FD001.txt', 'test_FD002.txt', 'RUL_FD003.txt', 'RUL_FD001.txt', 'test_FD004.txt', 'test_FD001.txt', 'test_FD003.txt', 'RUL_FD004.txt', 'RUL_FD002.txt', 'train_FD003.txt', 'train_FD002.txt', 'sample_data']


In [ ]:
## Importing important Library.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dropout, Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

## Reading all the "train_FD00[i].txt" files as Dataframes normaly.

train1=pd.read_csv('train_FD001.txt',sep="\s+",header=None)
train2=pd.read_csv('train_FD002.txt',sep="\s+",header=None)
train3=pd.read_csv('train_FD003.txt',sep="\s+",header=None)
train4=pd.read_csv('train_FD004.txt',sep="\s+",header=None)

## Providing the Feature names.

column_names=["engine", "cycle", "setting1", "setting2", "setting3", "sensor1", "sensor2", "sensor3", "sensor4", "sensor5", "sensor6", "sensor7", "sensor8", "sensor9", "sensor10", "sensor11", "sensor12", "sensor13", "sensor14", "sensor15", "sensor16", "sensor17", "sensor18", "sensor19", "sensor20", "sensor21"]

## Assigning the Feature names to the corresponding Dataframe.

train1.columns=column_names
train2.columns=column_names
train3.columns=column_names
train4.columns=column_names

## Unique categories of each feature in train Dataframes.

print(f"Unique categories of each feature in train1 Dataframe: {len(train1.columns)}\n\n{train1.nunique()}")
print(f"\nUnique categories of each feature in train2 Dataframe: {len(train2.columns)}\n\n{train2.nunique()}")
print(f"\nUnique categories of each feature in train3 Dataframe: {len(train3.columns)}\n\n{train3.nunique()}")
print(f"\nUnique categories of each feature in train4 Dataframe: {len(train4.columns)}\n\n{train4.nunique()}")

# Some of the feature's unique categories are 1.
#'setting3', 'sensor1', 'sensor5', 'sensor10', 'sensor16', 'sensor18' and 'sensor19' in train1
#'setting3', 'sensor1', 'sensor5', 'sensor16', 'sensor18', 'sensor19' in train3

## Removing Useless features from all selected Dataframes.
# These drops are based on constant values in the training sets, maintain consistency.
train1=train1.drop(columns=["setting3", "sensor1", "sensor5", "sensor10", "sensor16", "sensor18", "sensor19"])
train3=train3.drop(columns=["setting3", "sensor1", "sensor5", "sensor16", "sensor18", "sensor19"])

## Checking remaining Features all Dataframes.

print(f"Remaining features of 'train1' Dataframe: {len(train1.columns)}\n\n{train1.columns}")
print(f"\nRemaining features of 'train2' Dataframe: {len(train2.columns)}\n\n{train2.columns}")
print(f"\nRemaining features of 'train3' Dataframe: {len(train3.columns)}\n\n{train3.columns}")
print(f"\nRemaining features of 'train4' Dataframe: {len(train4.columns)}\n\n{train4.columns}")

## Defining the function to calculate the RUL.

def Calculate_RUL(df):
    max_cycles = df.groupby('engine')['cycle'].max()
    merged = df.merge(max_cycles.to_frame(name='max_time_cycle'), left_on='engine',right_index=True)
    merged["RUL"] = merged["max_time_cycle"] - merged['cycle']
    merged = merged.drop("max_time_cycle", axis=1)
    return merged

## Merging the calculated RUL in the train Dataframes.

train1=Calculate_RUL(train1)
train2=Calculate_RUL(train2)
train3=Calculate_RUL(train3)
train4=Calculate_RUL(train4)

## Splitting 'train1' Dataframe for model training.

X1=train1.drop(columns=['engine', 'cycle', 'setting1', 'setting2', 'sensor6', 'RUL'])
y1=train1['RUL']
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.20, random_state=42)

## Splitting 'train2' Dataframe for model training.
# Drop columns consistent with how train2 has constant values or deemed less relevant for FD002.
# Based on the previous error, X2 has 18 features after these drops.
X2=train2.drop(columns=['engine', 'cycle', 'setting1', 'setting2', 'setting3', 'sensor13', 'sensor16', 'sensor19', 'RUL']) # Removed 'cycle' from drops as it's not a feature for the model
y2=train2['RUL']
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.20, random_state=42)
print(f"Number of features in X_train2 before scaling: {X_train2.shape[1]}") # Should be 18

## Splitting 'train3' Dataframe for model training.

X3=train3.drop(columns=['engine', 'cycle', 'setting1', 'setting2', 'sensor6', 'sensor10', 'RUL'])
y3=train3['RUL']
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.20, random_state=42)

## Splitting 'train4' Dataframe for model training.

X4=train4.drop(columns=['engine', 'cycle', 'setting1', 'setting2', 'setting3', 'sensor13', 'sensor16', 'sensor19', 'RUL']) # Removed 'cycle' from drops
y4=train4['RUL']
X_train4, X_test4, y_train4, y_test4 = train_test_split(X4, y4, test_size=0.20, random_state=42)

## Importing RUL_FD001, RUL_FD002, RUL_FD003 and RUL_FD004.

rul1=pd.read_csv("RUL_FD001.txt",header=None, names=["RUL"])
rul2=pd.read_csv("RUL_FD002.txt",header=None, names=["RUL"])
rul3=pd.read_csv("RUL_FD003.txt",header=None, names=["RUL"])
rul4=pd.read_csv("RUL_FD004.txt",header=None, names=["RUL"])

## Importing test datasets.

test1=pd.read_csv('test_FD001.txt',sep="\s+",header=None)
test2=pd.read_csv('test_FD002.txt',sep="\s+",header=None)
test3=pd.read_csv('test_FD003.txt',sep="\s+",header=None)
test4=pd.read_csv('test_FD004.txt',sep="\s+",header=None)

## Assigning the Feature names to the corresponding Dataframe.

test1.columns=column_names
test2.columns=column_names
test3.columns=column_names
test4.columns=column_names

# Initialize a scaler for each dataset's training data.
# This ensures that test/validation data are scaled using the training set's min/max.
scaler1 = MinMaxScaler()
scaler2 = MinMaxScaler()
scaler3 = MinMaxScaler()
scaler4 = MinMaxScaler()

## Scaling X_train1 and X_test1
scaled_train1=scaler1.fit_transform(X_train1)
scaled_test1=scaler1.transform(X_test1) # Use transform, not fit_transform

## Conserve only the last occurence of each unit to match the length of test1 (Old logic - will be replaced)
# X_valid1 = test1.groupby('engine').last().reset_index().drop(columns=["engine", "cycle", "setting1", "setting2", "setting3"])
# X_valid1.drop(columns=['sensor1','sensor5','sensor6','sensor10','sensor16','sensor18','sensor19'], inplace=True)
# scaled_X_valid1=scaler1.transform(X_valid1) # Use transform

## Scaling X_train2 and X_test2
scaled_train2=scaler2.fit_transform(X_train2)
scaled_test2=scaler2.transform(X_test2) # Use transform
print(f"Number of features in scaled_train2: {scaled_train2.shape[1]}") # Should be 18

## Conserve only the last occurence of each unit to match the length of test2 (Old logic - will be replaced)
# X_valid2 = test2.groupby('engine').last().reset_index().drop(columns=["engine", "cycle", "setting1", "setting2", "setting3"])
# X_valid2.drop(columns=['sensor13','sensor16', 'sensor19'], inplace=True)
# scaled_X_valid2=scaler2.transform(X_valid2) # Use transform

## Scaling X_train3 and X_test3
scaled_train3=scaler3.fit_transform(X_train3)
scaled_test3=scaler3.transform(X_test3) # Use transform

## Conserve only the last occurence of each unit to match the length of test3 (Old logic - will be replaced)
# X_valid3 = test3.groupby('engine').last().reset_index().drop(columns=["engine", "cycle", "setting1", "setting2", "setting3"])
# X_valid3.drop(columns=['sensor1','sensor5','sensor6', 'sensor10', 'sensor16','sensor18','sensor19'], inplace=True)
# scaled_X_valid3=scaler3.transform(X_valid3) # Use transform

## Scaling X_train4 and X_test4
scaled_train4=scaler4.fit_transform(X_train4)
scaled_test4=scaler4.transform(X_test4) # Use transform

## Conserve only the last occurence of each unit to match the length of test4 (Old logic - will be replaced)
# X_valid4 = test4.groupby('engine').last().reset_index().drop(columns=["engine", "cycle", "setting1", "setting2", "setting3"])
# X_valid4.drop(columns=['sensor13','sensor16', 'sensor19'], inplace=True)
# scaled_X_valid4=scaler4.transform(X_valid4) # Use transform

# generate a sequence for LSTM model
def generate_sequences(data, labels, window_size):
    sequences = []
    targets = []
    # Ensure data is numpy array for slicing
    data = np.array(data)
    # Ensure labels is a Series or DataFrame for iloc, then convert to numpy at the end
    if not isinstance(labels, (pd.Series, pd.DataFrame)):
        labels = pd.Series(labels) # Convert to Series if it's already a numpy array

    for i in range(len(data) - window_size):
        sequence = data[i:i+window_size]
        target = labels.iloc[i + window_size]
        sequences.append(sequence)
        targets.append(target)
    return np.array(sequences), np.array(targets)

# New function to extract last 'window_size' cycles for test prediction
# This aligns test data preparation with the sequence length the model expects.
def get_last_sequences_for_test(df_test, window_size, features_to_extract, scaler_obj):
    sequences = []
    # Ensure df_test has the 'engine' column and others for grouping and extraction
    df_test_temp = df_test.copy() # Avoid modifying original

    for engine_id in df_test_temp['engine'].unique():
        engine_data = df_test_temp[df_test_temp['engine'] == engine_id][features_to_extract]

        # Check if engine_data is not empty
        if engine_data.empty:
            continue

        # Scale the engine-specific data using the pre-fitted scaler
        scaled_engine_data = scaler_obj.transform(engine_data)

        if len(scaled_engine_data) >= window_size:
            # Take the last 'window_size' observations
            sequences.append(scaled_engine_data[-window_size:])
        else:
            # If an engine has fewer cycles than window_size, pad it
            # Common padding strategies:
            # 1. Replicate the first observation:
            # padded_sequence = np.tile(scaled_engine_data[0], (window_size, 1))
            # padded_sequence[-len(scaled_engine_data):] = scaled_engine_data
            # 2. Replicate the last observation (more common for RUL):
            padded_sequence = np.tile(scaled_engine_data[-1], (window_size, 1))
            padded_sequence[:len(scaled_engine_data)] = scaled_engine_data
            sequences.append(padded_sequence)
            # Alternative: pad with zeros or skip this engine if window_size is strict
    return np.array(sequences)


# Our model building function
def build_model(input_features, window_size=30): # Pass the number of features and window_size as argument
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(window_size, input_features)),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),  # Optional: Downsamples feature map
        Dropout(0.3),

        LSTM(64, return_sequences=True),
        Dropout(0.3),

        LSTM(32),
        Dropout(0.3),

        Dense(32, activation='relu'),
        Dense(1)  # Final regression output
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model # Return the compiled model

BATCH_SIZE = 64
EPOCHS = 100
WINDOW_SIZE = 30 # Define window size globally

# Train the model
def train_model(model, X_train, y_train, X_val=None, y_val=None, batch_size=BATCH_SIZE, epochs=EPOCHS):
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, min_lr=0.00001) # Added ReduceLROnPlateau
    ]

    if X_val is not None:
        history = model.fit(X_train, y_train,
                            validation_data=(X_val, y_val),
                            batch_size=batch_size,
                            epochs=epochs,
                            callbacks=callbacks)
    else:
        history = model.fit(X_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            callbacks=callbacks)

    return history

# Evaluate model (modified to calculate metrics manually)
def evaluate_model_manual(model, X, y, label=""):
    preds = model.predict(X, verbose=0) # Set verbose to 0 to suppress per-batch output

    if hasattr(preds, "numpy"):
        preds = preds.numpy()

    preds = preds.reshape(-1) # Flatten predictions to 1D

    # Ensure y is also a 1D numpy array
    if isinstance(y, (pd.Series, pd.DataFrame)):
        y_true = y.values.reshape(-1)
    else:
        y_true = y.reshape(-1)

    mse = mean_squared_error(y_true, preds)
    mae = mean_absolute_error(y_true, preds)
    r2 = r2_score(y_true, preds) # Optional: Add R2 score

    print(f"{label} -> MSE: {mse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

    print(f"{label} preds type:", type(preds), "shape:", preds.shape)
    print(f"{label} true_y type:", type(y_true), "shape:", y_true.shape)

    return preds

# --- Focus on FD002 Dataset ---

# Determine the number of features for FD002 model input
# X_train2 has 18 features (columns)
fd002_input_features = X_train2.shape[1]
print(f"FD002 Model Input Features: {fd002_input_features}")

# Build the model for FD002
model_fd002 = build_model(fd002_input_features, window_size=WINDOW_SIZE)
model_fd002.summary()

# Apply to FD002 Dataset and prepare sequences for training and internal validation
X_seq_train2, y_seq_train2 = generate_sequences(scaled_train2, y_train2, window_size=WINDOW_SIZE)
X_seq_test2, y_seq_test2 = generate_sequences(scaled_test2, y_test2, window_size=WINDOW_SIZE) # For internal validation during training

# Confirm sequence shapes
print("X_seq_train2 shape:", X_seq_train2.shape)
print("y_seq_train2 shape:", y_seq_train2.shape)
print("X_seq_test2 shape:", X_seq_test2.shape)
print("y_seq_test2 shape:", y_seq_test2.shape)
print("Types: X_seq_train2 - {}, y_seq_train2 - {}".format(type(X_seq_train2), type(y_seq_train2)))

# Train on FD002
print("\n--- Training Model for FD002 ---")
# Use X_seq_test2 and y_seq_test2 for validation during training
history_fd002 = train_model(model_fd002, X_seq_train2, y_seq_train2, X_val=X_seq_test2, y_val=y_seq_test2)

# Predict sample and print shape
sample_pred_fd002 = model_fd002.predict(X_seq_train2[:10])
print("Sample Prediction Shape (after model_fd002.predict):", sample_pred_fd002.shape)


# --- Prepare Test Data (FD002) for Final Evaluation ---
# Define features to extract from the raw test2 dataframe, consistent with X2
# These are the columns that were NOT dropped in X2, excluding 'engine', 'cycle', 'RUL'
features_for_test2_final_eval = [col for col in column_names if col not in ['engine', 'cycle', 'setting1', 'setting2', 'setting3', 'sensor13', 'sensor16', 'sensor19']]

# Use the scaler that was fitted on X_train2
X_valid2_final_sequences = get_last_sequences_for_test(test2, window_size=WINDOW_SIZE,
                                                         features_to_extract=features_for_test2_final_eval,
                                                         scaler_obj=scaler2)
y_valid2_final = rul2['RUL'].values # The true RULs for the test engines


# Confirm shapes for final evaluation
print("X_valid2_final_sequences shape (for final evaluation):", X_valid2_final_sequences.shape)
print("y_valid2_final shape (for final evaluation):", y_valid2_final.shape)


# Evaluate model using the manual evaluation function
print("\n--- Evaluation on FD002 ---")
train_preds_fd002 = evaluate_model_manual(model_fd002, X_seq_train2, y_seq_train2, label="Train Set (FD002)")
test_preds_fd002  = evaluate_model_manual(model_fd002, X_seq_test2,  y_seq_test2,  label="Test Set (FD002 - Internal Validation)")
final_valid_preds_fd002 = evaluate_model_manual(model_fd002, X_valid2_final_sequences, y_valid2_final, label="Final Validation Set (FD002 - RUL2)")

Unique categories of each feature in train1 Dataframe: 26

engine       100
cycle        362
setting1     158
setting2      13
setting3       1
sensor1        1
sensor2      310
sensor3     3012
sensor4     4051
sensor5        1
sensor6        2
sensor7      513
sensor8       53
sensor9     6403
sensor10       1
sensor11     159
sensor12     427
sensor13      56
sensor14    6078
sensor15    1918
sensor16       1
sensor17      13
sensor18       1
sensor19       1
sensor20     120
sensor21    4745
dtype: int64

Unique categories of each feature in train2 Dataframe: 26

engine        260
cycle         378
setting1      536
setting2      105
setting3        2
sensor1         6
sensor2      1590
sensor3     12305
sensor4     15411
sensor5         6
sensor6        14
sensor7      2067
sensor8       897
sensor9     22434
sensor10        9
sensor11      681
sensor12     1672
sensor13      514
sensor14    14905
sensor15     8464
sensor16        2
sensor17       53
sensor18        6
sensor19    

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_5 (Conv1D)               │ (None, 28, 64)         │         3,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 28, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 14, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 14, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ (None, 14, 64)         │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 14, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,305 (196.50 KB)

 Trainable params: 50,177 (196.00 KB)

 Non-trainable params: 128 (512.00 B)

X_seq_train2 shape: (42977, 30, 18)
y_seq_train2 shape: (42977,)
X_seq_test2 shape: (10722, 30, 18)
y_seq_test2 shape: (10722,)
Types: X_seq_train2 - <class 'numpy.ndarray'>, y_seq_train2 - <class 'numpy.ndarray'>

--- Training Model for FD002 ---
Epoch 1/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 14s 13ms/step - loss: 10158.0811 - mae: 79.6838 - val_loss: 4694.3936 - val_mae: 56.5061 - learning_rate: 0.0010
Epoch 2/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 4981.0903 - mae: 58.0094 - val_loss: 4696.5068 - val_mae: 56.4779 - learning_rate: 0.0010
Epoch 3/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - loss: 4985.1279 - mae: 57.9885 - val_loss: 4695.3838 - val_mae: 56.6377 - learning_rate: 0.0010
Epoch 4/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 4960.8335 - mae: 57.8404 - val_loss: 4702.8179 - val_mae: 56.4415 - learning_rate: 0.0010
Epoch 5/100
672/672 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - loss: 4968.1138 - mae: 58.0793 - val_loss: 4696.2505 - val_mae: 56.4806 - learning_rat